In [ ]:
%load_ext autoreload
%autoreload 2

# Project 2 - Part 2
For Part 2, attempt to fine-tune the question-answering with your questions and answers intended to offer greater resolution than the answers in Part 1.
To do this, you will need to generate your own custom SQuAD-compatible QA dataset using the sections of your choice, the questions, and the answers which you will add to the squad dataset downloadable from Hugging Face, [which is described in this article](https://huggingface.co/transformers/v3.2.0/custom_datasets.html).  Note: custom training can take a very long time on Google Colab, so be prepared for that.